In [82]:
from pickhardtpayments.Payment import logger
from pickhardtpayments.ChannelGraph import ChannelGraph
from pickhardtpayments.UncertaintyNetwork import UncertaintyNetwork
from pickhardtpayments.OracleLightningNetwork import OracleLightningNetwork
from pickhardtpayments.SyncSimulatedPaymentSession import SyncSimulatedPaymentSession
import networkx as nx
import ndjson
import json

In [83]:
channel_graph = ChannelGraph("../channels.sample.json")
uncertainty_network = UncertaintyNetwork(channel_graph)
oracle_lightning_network = OracleLightningNetwork(channel_graph)

In [84]:
payment_set = ndjson.load(open("./payments_small_graph.ndjson", "r"))

In [85]:
# init simulation
c = 0
successful_payments = 0
failed_payments = 0
payment_simulation = []

# create new payment session - will later be moved before payment loop to simulate sequence of payments
sim_session = SyncSimulatedPaymentSession(oracle_lightning_network, uncertainty_network, prune_network=False)

In [86]:
for payment in payment_set:
    c += 1
    # create new payment session
    # sim_session = SyncSimulatedPaymentSession(oracle_lightning_network, uncertainty_network, prune_network=False)
    # we need to make sure we forget all learnt information on the Uncertainty Network
    # sim_session.forget_information()
    ret = sim_session.pickhardt_pay(payment["sender"], payment["receiver"], payment["amount"],
                                    mu=0, base=0, loglevel="warning")
    if ret > 0:
        successful_payments += 1
    if ret < 0:
        failed_payments += 1
    payment['success'] = ret
    payment_simulation.append(payment)

In [87]:
print(f"\n{c} payments. {successful_payments} successful, {failed_payments} failed.")


20 payments. 17 successful, 3 failed.


In [88]:
oracle_lightning_network_prior = OracleLightningNetwork(channel_graph)

In [89]:
for edge in oracle_lightning_network.network.edges:
    channel_init = oracle_lightning_network_prior.get_channel(edge[0], edge[1], edge[2])
    channel_now = oracle_lightning_network.get_channel(edge[0], edge[1], edge[2])
    print(f"{edge[0]}-{edge[1]}: {channel_init.actual_liquidity:>8.0f} to {channel_now.actual_liquidity:>8.0f}. "
          f"Intervall [{uncertainty_network.get_channel(edge[0], edge[1], edge[2]).min_liquidity}; "
          f"{uncertainty_network.get_channel(edge[0], edge[1], edge[2]).max_liquidity}]")

A-B:    52500 to    52500. Intervall [0; 105000]
A-C:   102500 to    32500. Intervall [10000; 205000]
A-D:    52500 to   102500. Intervall [95000; 95000]
B-A:    52500 to    52500. Intervall [0; 105000]
B-C:    52500 to   102500. Intervall [95000; 95000]
B-D:   152500 to    92500. Intervall [10000; 295000]
C-B:    52500 to     2500. Intervall [10000; 10000]
C-A:   102500 to   172500. Intervall [0; 195000]
D-A:    52500 to     2500. Intervall [10000; 10000]
D-B:   152500 to   212500. Intervall [10000; 295000]
